In [5]:
from collections import Counter
from collections import defaultdict
import numpy as np
import pandas as pd
import spacy
from nltk import word_tokenize

from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

In [2]:
# load data
data = pd.read_csv('SEM-2012-SharedTask-CD-SCO-training-simple.v2.txt', sep="\t", header=None)
data.columns = ['annotator', 'sentence_id', 'token_id', 'token', 'label']

In [3]:
# make lower 
data["token_lower"] = data["token"].str.lower()
nlp = spacy.load("en_core_web_sm")
spacy_pipe = nlp.pipe(data["token_lower"].values, disable=["ner", "parser"])
# get lemma, pos, previous lemma, previous pos, next lemma, next pos 
features_gen = ((doc[0].lemma_, doc[0].pos_) for doc in spacy_pipe)
data["lemma"], data["pos"] = zip(*features_gen)
data["prev_Lemma"] = data["lemma"].shift(periods=1)
data["next_Lemma"] = data["lemma"].shift(periods=-1)
data["prev_pos"] = data["pos"].shift(periods=1)
data["next_pos"] = data["pos"].shift(periods=-1)

In [7]:
# trying some things out with stemmers 
snow = SnowballStemmer(language='english')
data["snowballStemmer"] = data.apply(lambda row: snow.stem(row["token_lower"]), axis=1)
port = PorterStemmer()
data['PorterStemmer'] = data.apply(lambda row: port.stem(row["token_lower"]), axis=1)


In [8]:
data

,annotator,sentence_id,token_id,token,label,token_lower,lemma,pos,prev_Lemma,next_Lemma,prev_pos,next_pos,snowballStemmer,PorterStemmer
0,baskervilles01,0,0,Chapter,O,chapter,chapter,NOUN,NaN,1,NaN,X,chapter,chapter
1,baskervilles01,0,1,1.,O,1.,1,X,chapter,mr,NOUN,NOUN,1.,1.
2,baskervilles01,0,2,Mr.,O,mr.,mr,NOUN,1,sherlock,X,VERB,mr.,mr.
3,baskervilles01,0,3,Sherlock,O,sherlock,sherlock,VERB,mr,holme,NOUN,NOUN,sherlock,sherlock
4,baskervilles01,0,4,Holmes,O,holmes,holme,NOUN,sherlock,mr,VERB,NOUN,holm,holm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65446,baskervilles14,270,58,slopes,O,slopes,slope,NOUN,russet,of,VERB,ADP,slope,slope
65447,baskervilles14,270,59,of,O,of,of,ADP,slope,the,NOUN,DET,of,of
65448,baskervilles14,270,60,the,O,the,the,DET,of,moor,ADP,NOUN,the,the
65449,baskervilles14,270,61,moor,O,moor,moor,NOUN,the,.,DET,PUNCT,moor,moor
